In [ ]:
from sklearn.feature_selection import f_classif
import arff
import seaborn as sns
import matplotlib.pyplot as plt

def main():

    with open("HW1/diabetes.arff", "r") as file:

        dataset = arff.load(file)
        attribute_names = [attr[0] for attr in dataset['attributes']]
        data_dict = {attr: [] for attr in attribute_names}

        for row in dataset['data']:
            for i, value in enumerate(row):
                data_dict[attribute_names[i]].append(value)

        print(data_dict)

main()